In [27]:
import os
from PIL import Image
from tqdm.auto import tqdm

c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
xml_dir = './source/annotations'
xml_names = os.listdir(xml_dir)
xml_paths = [f'{xml_dir}/{xml_name}' for xml_name in xml_names if xml_name.split('.')[-1] == 'xml']

In [12]:
xml_paths

['./source/annotations/C100_D1_P1.xml',
 './source/annotations/C100_D1_P2.xml',
 './source/annotations/C100_D1_P3.xml',
 './source/annotations/C100_D1_P4.xml',
 './source/annotations/C100_D2_P1.xml',
 './source/annotations/C100_D2_P2.xml',
 './source/annotations/C100_D2_P3.xml',
 './source/annotations/C100_D2_P4.xml',
 './source/annotations/C101_D1_P1.xml',
 './source/annotations/C101_D1_P2.xml',
 './source/annotations/C101_D1_P3.xml',
 './source/annotations/C101_D1_P4.xml',
 './source/annotations/C101_D2_P1.xml',
 './source/annotations/C101_D2_P2.xml',
 './source/annotations/C101_D2_P3.xml',
 './source/annotations/C101_D2_P4.xml',
 './source/annotations/C102_D1_P1.xml',
 './source/annotations/C102_D1_P2.xml',
 './source/annotations/C102_D1_P3.xml',
 './source/annotations/C102_D1_P4.xml',
 './source/annotations/C102_D2_P1.xml',
 './source/annotations/C102_D2_P2.xml',
 './source/annotations/C102_D2_P3.xml',
 './source/annotations/C102_D2_P4.xml',
 './source/annotations/C103_D1_P1.xml',


In [13]:
import xmltodict
xml_path = './source/annotations/C1_D1_P1.xml'
with open(xml_path, 'r') as file:
   xml_string = ' '.join(file.readlines())

xml_parsed = xmltodict.parse(xml_string)

In [14]:
xml_parsed

{'annotation': {'folder': 'images',
  'filename': 'C1_D1_P1.jpg',
  'path': './drafter_1/images/C1_D1_P1.jpg',
  'source': {'database': 'CGHD'},
  'size': {'width': '960', 'height': '1280', 'depth': '3'},
  'segmented': '0',
  'object': [{'name': 'voltage.dc',
    'pose': 'Unspecified',
    'truncated': '0',
    'difficult': '0',
    'bndbox': {'xmin': '105', 'ymin': '295', 'xmax': '197', 'ymax': '389'}},
   {'name': 'transistor.bjt',
    'pose': 'Unspecified',
    'truncated': '0',
    'difficult': '0',
    'bndbox': {'xmin': '321', 'ymin': '286', 'xmax': '370', 'ymax': '359'}},
   {'name': 'resistor',
    'pose': 'Unspecified',
    'truncated': '0',
    'difficult': '0',
    'bndbox': {'xmin': '352',
     'ymin': '386',
     'xmax': '384',
     'ymax': '449',
     'rotation': '90'}},
   {'name': 'diode.light_emitting',
    'pose': 'Unspecified',
    'truncated': '0',
    'difficult': '0',
    'bndbox': {'xmin': '356',
     'ymin': '460',
     'xmax': '421',
     'ymax': '505',
     '

In [15]:
CLASSES = {
   '__background__':   0,
    'text':         1,
    'junction':     2,
    'crossover':    3,
    'terminal':     4,
    'gnd':          5,
    'vss':          6,
    'voltage.dc':       7,
    'voltage.ac':       8,
    'voltage.battery':  9,
    'resistor':             10,
    'resistor.adjustable':  11,
    'resistor.photo':       12,
    'capacitor.unpolarized':    13,
    'capacitor.polarized':      14,
    'capacitor.adjustable':     15,
    'inductor':         16,
    'inductor.ferrite': 17,
    'inductor.coupled': 18,
    'transformer':      19,
    'diode':                20,
    'diode.light_emitting': 21,
    'diode.thyrector':      22,
    'diode.zener':          23,
    'diac':                 24,
    'triac':                25,
    'thyristor':            26,
    'varistor':             27,
    'transistor.bjt':   28,
    'transistor.fet':   29,
    'transistor.photo': 30,
    'operational_amplifier':                    31,
    'operational_amplifier.schmitt_trigger':    32,
    'optocoupler':                              33,
    'integrated_circuit':                   34,
    'integrated_circuit.ne555':             35,
    'integrated_circuit.voltage_regulator': 36,
    'xor':  37,
    'and':  38,
    'or':   39,
    'not':  40,
    'nand': 41,
    'nor':  42,
    'probe.current': 43,
    'probe.voltage': 44,
    'switch':   45,
    'relay':    46,
    'socket':   47,
    'fuse':     48,
    'speaker':      49,
    'motor':        50,
    'lamp':         51,
    'microphone':   52,
    'antenna':      53,
    'crystal':      54,
    'mechanical':   55,
    'magnetic':     56,
    'optical':      57,
    'block':        58,
    'unknown':  59
}

In [16]:
def get_annotation(xml, img_path):
   with Image.open(img_path) as im:
      x_img_length, y_img_length = im.width, im.height

   elements = xml['annotation']['object']

   annotation_str = ''

   if isinstance(elements, dict):
      elements = [elements]
   for elem in elements:
      elem_class = CLASSES[elem['name']]

      x_box_length = float(elem['bndbox']['xmax']) - float(elem['bndbox']['xmin'])
      y_box_length = float(elem['bndbox']['ymax']) - float(elem['bndbox']['ymin'])
      x_box_center = float(elem['bndbox']['xmin']) + (x_box_length/2)
      y_box_center = float(elem['bndbox']['ymin']) + (y_box_length/2)

      x_box_length = x_box_length / x_img_length
      y_box_length = y_box_length / y_img_length

      x_box_center = x_box_center / x_img_length
      y_box_center = y_box_center / y_img_length

      if x_box_length > 1:
         x_box_length = 1.0
      if y_box_length > 1:
         y_box_length = 1.0
      if x_box_center > 1:
         x_box_center = 1.0
      if y_box_center > 1:
         y_box_center = 1.0

      annotation_str += f'{elem_class} {x_box_center} {y_box_center} {x_box_length} {y_box_length}\n'

   return annotation_str

In [17]:
#yolo_annotation = get_annotation(xml_parsed)
#print(yolo_annotation)

In [18]:
#with open(xml_path.replace('xml', 'txt'), 'w') as file:
#   file.write(yolo_annotation)

In [35]:
img_names = os.listdir('./source/images/')

for xml_path in tqdm(xml_paths):
   with open(xml_path, 'r') as file:
      xml_string = ' '.join(file.readlines())

   xml_parsed = xmltodict.parse(xml_string)
   img_path = xml_path.replace('xml', 'jpg').replace('annotations', 'images')
   if not os.path.exists(img_path):
      img_name = next(name for name in img_names 
                      if os.path.basename(img_path).split('.')[0] in name)
      img_path = os.path.join(os.path.dirname(img_path), img_name)
   yolo_annotation = get_annotation(xml_parsed, img_path)
   with open(xml_path.replace('xml', 'txt'), 'w') as file:
      file.write(yolo_annotation)

100%|██████████| 2208/2208 [03:13<00:00, 11.39it/s]


In [33]:
os.path.basename(img_path).split('.')[0]

'C169_D1_P1'